# Combine the player data with the Madden data

In [129]:
import pandas as pd

In [130]:
madden_df = pd.read_csv("./data2023/actualmaddenratings.csv")
players_df = pd.read_csv("./data2023/players.csv")

In [131]:
print(madden_df.head())

   #     Team Position          Archetype  Jersey # Unnamed: 5 Last Name  Age  \
0  1  Packers       WR     WR_RouteRunner        17    Davante     Adams   28   
1  2   Chiefs       QB      QB_Improviser        15    Patrick   Mahomes   25   
2  3   Chiefs       TE  TE_VerticalThreat        87     Travis     Kelce   31   
3  4     Rams       RE     DE_PowerRusher        99      Aaron    Donald   30   
4  5     Rams       CB        CB_MantoMan         5      Jalen    Ramsey   26   

   Height  Weight  ...  Years Pro  Handedness        College  Portrait ID  \
0      73     215  ...          7       Right   Fresno State         6328   
1      75     230  ...          4       Right     Texas Tech         8648   
2      77     260  ...          8       Right     Cincinnati         5920   
3      73     280  ...          7       Right     Pittsburgh         6394   
4      73     208  ...          5       Right  Florida State         8185   

   Team ID  Primary Key        Full Name          

In [132]:
print(players_df.head(15))

    nflId height  weight   birthDate      collegeName officialPosition  \
0   25511    6-4     225  1977-08-03         Michigan               QB   
1   28963    6-5     240  1982-03-02        Miami, O.               QB   
2   29550    6-4     328  1982-01-22         Arkansas                T   
3   29851    6-2     225  1983-12-02       California               QB   
4   30078    6-2     228  1982-11-24          Harvard               QB   
5   30842    6-6     267  1984-05-19             UCLA               TE   
6   30869    6-7     330  1981-12-12  Louisiana State                T   
7   33084    6-4     217  1985-05-17   Boston College               QB   
8   33107    6-4     315  1985-08-30    Virginia Tech                T   
9   33130   5-10     175  1986-12-01       California               WR   
10  33131    6-8     300  1986-09-01            Miami               DE   
11  33138    6-3     222  1985-07-02         Michigan               QB   
12  33241    6-3     219  05/15/1986  

In [133]:
# Ensure all birthdates use the same formatting
players_df['birthDate'] = pd.to_datetime(players_df['birthDate'])
madden_df['Birthdate'] = pd.to_datetime(madden_df['Birthdate'])
players_df['birthDate'] = players_df['birthDate'].dt.strftime('%d/%m/%Y')
madden_df['Birthdate'] = madden_df['Birthdate'].dt.strftime('%d/%m/%Y')

print(players_df['birthDate'])
print(madden_df['Birthdate'])

0       03/08/1977
1       02/03/1982
2       22/01/1982
3       02/12/1983
4       24/11/1982
           ...    
1674           NaN
1675           NaN
1676           NaN
1677           NaN
1678           NaN
Name: birthDate, Length: 1679, dtype: object
0       24/12/1992
1       17/09/1995
2       10/05/1989
3       23/05/1991
4       24/10/1994
           ...    
2075    10/07/1994
2076    21/08/1997
2077    26/01/1996
2078    26/09/1998
2079    15/07/1994
Name: Birthdate, Length: 2080, dtype: object


C:\Users\ellio\AppData\Local\Temp\ipykernel_4664\903581096.py:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  madden_df['Birthdate'] = pd.to_datetime(madden_df['Birthdate'])


# Add a column for names from the Madden data which include suffixes (e.g. Jr)

In [134]:
players_names_list = players_df["displayName"]
madden_names_list = madden_df["Full Name"]

In [135]:
count = 0
updated_players_names = list(players_names_list)
for name in players_names_list:
    i = 0
    while i < len(madden_names_list):
        if name in madden_names_list[i]:
            updated_players_names[count] = madden_names_list[i]
            i = 10000
        i += 1
    count += 1

print(len(updated_players_names))
#print(updated_players_names)

1679


In [136]:
players_df['updatedName'] = updated_players_names
print(players_df.head())

   nflId height  weight   birthDate collegeName officialPosition  \
0  25511    6-4     225  03/08/1977    Michigan               QB   
1  28963    6-5     240  02/03/1982   Miami, O.               QB   
2  29550    6-4     328  22/01/1982    Arkansas                T   
3  29851    6-2     225  02/12/1983  California               QB   
4  30078    6-2     228  24/11/1982     Harvard               QB   

          displayName         updatedName  
0           Tom Brady           Tom Brady  
1  Ben Roethlisberger  Ben Roethlisberger  
2        Jason Peters        Jason Peters  
3       Aaron Rodgers       Aaron Rodgers  
4    Ryan Fitzpatrick    Ryan Fitzpatrick  


# Now include those with slightly different punctuation

In [137]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
import string

players_df["nopunctuationname"] = players_df["updatedName"].apply(remove_punctuations)
madden_df["nopunctuationname"] = madden_df["Full Name"].apply(remove_punctuations)

In [138]:
madden_names_list = madden_df["Full Name"]
updated_players_names_list = players_df["updatedName"]
madden_nopunc = madden_df["nopunctuationname"]
players_nopunc = players_df["nopunctuationname"]
count = 0
updated_players_names2 = list(players_names_list)
for name in players_nopunc:
    i = 0
    while i < len(madden_names_list):
        if name in madden_nopunc[i]:
            updated_players_names2[count] = madden_names_list[i]
            i = 10000
        i += 1
    count += 1

In [139]:
players_df['updatedName2'] = updated_players_names2
players_df.to_csv('trialplayers.csv')  # Use to check file if necessary

In [140]:
# madden_df.loc[madden_df["Full Name"].isin(updated_players_names)]

# Replace NA dates using the Madden data

In [141]:
players_date = players_df['birthDate']
madden_date = madden_df['Birthdate']
players_names = players_df['updatedName2']
madden_names = madden_df["Full Name"]
players_pos = players_df['officialPosition']
madden_pos = madden_df['Position']
count = 0
for date in players_date:
    if pd.DataFrame([date]).isnull()[0][0] == True:
        player = players_names[count]
        i = 0
        while i < len(madden_names):
            if player in madden_names[i]:
                if player == 'Michael Carter':  # Michael Carter is the only duplicate name which has a nan in the date column
                    if players_pos[count] == madden_pos[i]:
                        players_date[count] = madden_date[i]
                else:
                    players_date[count] = madden_date[i]
            i += 1
        
    count += 1

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


C:\Users\ellio\AppData\Local\Temp\ipykernel_4664\2052361875.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_date[count] = madden_date[i]


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


C:\Users\ellio\AppData\Local\Temp\ipykernel_4664\2052361875.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_date[count] = madden_date[i]


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [143]:
print(players_df['birthDate'])
players_df.to_csv('trialnewdate.csv')

0       03/08/1977
1       02/03/1982
2       22/01/1982
3       02/12/1983
4       24/11/1982
           ...    
1674           NaN
1675           NaN
1676    03/08/1998
1677           NaN
1678           NaN
Name: birthDate, Length: 1679, dtype: object


# Combine the two datasets using the D.O.B and Full name

First we need to make sure the column names match up (rename column titles)

In [144]:
madden_df.rename(columns={"Birthdate": "birthDate"}, inplace=True)
madden_df.rename(columns={"Full Name": "updatedName2"}, inplace=True)
print(madden_df.columns)
print(players_df.columns)

Index(['#', 'Team', 'Position', 'Archetype', 'Jersey #', 'Unnamed: 5',
       'Last Name', 'Age', 'Height', 'Weight', 'Overall', 'Speed',
       'Acceleration', 'Agility', 'Change of Dir', 'Strength', 'Jumping',
       'Awareness', 'Carrying', 'Break Tackle', 'Juke Move', 'Spin Move',
       'Trucking', 'Stiff Arm', 'BC Vision', 'Catching', 'Catch In Traffic',
       'Spec Catch', 'Release', 'Short RR', 'Medium RR', 'Deep RR',
       'Throw Power', 'Throw Acc Short', 'Throw Acc Mid', 'Throw Acc Deep',
       'Throw Under Pressure', 'Throw On The Run', 'Play Action', 'Break Sack',
       'Run Block', 'Run Block Power', 'Run Block Finesse', 'Pass Block',
       'Pass Block Power', 'Pass Block Finesse', 'Impact Blocking',
       'Lead Blocking', 'Tackle', 'Hit Power', 'Pursuit', 'Man Coverage',
       'Zone Coverage', 'Press', 'Play Recognition', 'Power Moves',
       'Finesse Moves', 'Block Shedding', 'Kick Power', 'Kick Accuracy',
       'Kick Return', 'Stamina', 'Injury', 'Toughness', 

Now we can combine the two datasets

In [145]:
merged_df = players_df.merge(madden_df, on=['birthDate','updatedName2']).sort_index(axis=1)
print(merged_df.columns)

Index(['#', 'Acceleration', 'Age', 'Agility', 'Archetype', 'Awareness',
       'BC Vision', 'Block Shedding', 'Break Sack', 'Break Tackle', 'Carrying',
       'Catch In Traffic', 'Catching', 'Change of Dir', 'College', 'Deep RR',
       'Finesse Moves', 'Handedness', 'Height', 'Hit Power', 'Impact Blocking',
       'Injury', 'Iteration', 'Jersey #', 'Juke Move', 'Jumping',
       'Kick Accuracy', 'Kick Power', 'Kick Return', 'Last Name',
       'Lead Blocking', 'Man Coverage', 'Medium RR', 'Overall', 'Pass Block',
       'Pass Block Finesse', 'Pass Block Power', 'Play Action',
       'Play Recognition', 'Player Asset', 'Portrait ID', 'Position',
       'Power Moves', 'Press', 'Primary Key', 'Pursuit', 'Release',
       'Run Block', 'Run Block Finesse', 'Run Block Power', 'Run Style',
       'Short RR', 'Signing Bonus', 'Spec Catch', 'Speed', 'Spin Move',
       'Stamina', 'Status', 'Stiff Arm', 'Strength', 'Tackle', 'Team',
       'Team ID', 'Throw Acc Deep', 'Throw Acc Mid', 'Throw Ac

In [146]:
merged_df = merged_df.drop(['nopunctuationname_x', 'nopunctuationname_y', '#'], axis=1)
print(merged_df.columns)
merged_df.to_csv('merged_df.csv') 

Index(['Acceleration', 'Age', 'Agility', 'Archetype', 'Awareness', 'BC Vision',
       'Block Shedding', 'Break Sack', 'Break Tackle', 'Carrying',
       'Catch In Traffic', 'Catching', 'Change of Dir', 'College', 'Deep RR',
       'Finesse Moves', 'Handedness', 'Height', 'Hit Power', 'Impact Blocking',
       'Injury', 'Iteration', 'Jersey #', 'Juke Move', 'Jumping',
       'Kick Accuracy', 'Kick Power', 'Kick Return', 'Last Name',
       'Lead Blocking', 'Man Coverage', 'Medium RR', 'Overall', 'Pass Block',
       'Pass Block Finesse', 'Pass Block Power', 'Play Action',
       'Play Recognition', 'Player Asset', 'Portrait ID', 'Position',
       'Power Moves', 'Press', 'Primary Key', 'Pursuit', 'Release',
       'Run Block', 'Run Block Finesse', 'Run Block Power', 'Run Style',
       'Short RR', 'Signing Bonus', 'Spec Catch', 'Speed', 'Spin Move',
       'Stamina', 'Status', 'Stiff Arm', 'Strength', 'Tackle', 'Team',
       'Team ID', 'Throw Acc Deep', 'Throw Acc Mid', 'Throw Acc Sho

In [147]:
merged_df2 = players_df.merge(madden_df, on=['updatedName2']).sort_index(axis=1)
merged_df2.to_csv('merged_df2.csv') 

In [148]:
merged_df3 = players_df.merge(madden_df, on=['birthDate']).sort_index(axis=1)
merged_df3.to_csv('merged_df3.csv') 